In [1]:
# Import a Kalman filter and other useful libraries
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import poly1d



In [221]:
Data=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_total.csv",index_col='Date')

In [277]:
df_cie=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Liste_entreprise.csv")
df_cie=df_cie.iloc[:, 1:]

logo=df_cie['logo'].tolist()
logo_pred=df_cie['logo_pred'].tolist()

df_cie.head(2)

,logo,logo_pred
0,AAPL,AAPL_Pred
1,ABT,ABT_Pred


# Kalman

In [27]:
# Construct a Kalman filter
kf = KalmanFilter(transition_matrices = [1],
                  observation_matrices = [1],
                  initial_state_mean = 0,
                  initial_state_covariance = 1,
                  observation_covariance=1,
                  transition_covariance=.01)


In [44]:
# Use the observed values of the price to get a rolling mean

df_kelman=pd.DataFrame()


for l in logo:
    state_means, _ = kf.filter(Data[l])
    state_means = pd.Series(state_means.flatten(), index=x.index)
    df_kelman[l]=Data[l]
    df_kelman[l+'_Pred']=state_means

In [52]:
df_kelman.tail(3)

,AAPL,AAPL_Pred,ABT,ABT_Pred,ACN,ACN_Pred,ATVI,ATVI_Pred,ADBE,ADBE_Pred,...,AAL,AAL_Pred,BLK,BLK_Pred,CBS,CBS_Pred,MSFT,MSFT_Pred,FB,FB_Pred
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,100.70,108.346263,42.56,44.451808,102.16,104.520056,36.79,38.195728,91.02,92.811875,...,41.23,42.448046,330.16,337.739216,46.13,47.195539,54.05,55.135631,102.97,104.664284
2016-01-07,96.45,107.214632,41.54,44.174823,99.16,104.010181,36.27,38.012544,89.11,92.459735,...,40.45,42.257982,315.73,335.645591,45.34,47.019031,52.17,54.853526,97.92,104.022735
2016-01-08,96.96,106.239161,40.67,43.841427,98.20,103.457488,35.71,37.793514,87.85,92.021234,...,40.37,42.078388,307.78,332.994879,46.46,46.965853,52.33,54.613475,97.33,103.386089


In [53]:
df_kelman.tail(3).to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Kalman_All_and_Pred.csv")

In [48]:
# Plot original data and estimated mean
#plt.figure(num=None, figsize=(30, 20), dpi=80)
#plt.plot(state_means)
#plt.plot()
#plt.title('Kalman filter estimate of average')
#plt.legend(['Kalman Estimate', 'X'])
#plt.xlabel('Day')
#plt.ylabel('Price');

In [47]:
#plt.figure(num=None, figsize=(30, 20), dpi=80, facecolor='w', edgecolor='k')
#plt.plot(state_means[-200:])
#plt.plot(x['Close'][-200:])
#
#plt.title('Kalman filter estimate of average')
#plt.legend(['Kalman Estimate', 'X'])
#plt.xlabel('Day')
#plt.ylabel('Price');

# ARIMA

In [ ]:
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

In [61]:
#get data
def GetData(fileName):
    return read_csv(fileName, header=0, parse_dates=[0], index_col=0).values

#Function that calls ARIMA model to fit and forecast the data
def StartARIMAForecasting(Actual, P, D, Q):
	model = ARIMA(Actual, order=(P, D, Q))
	model_fit = model.fit(disp=0)
	prediction = model_fit.forecast()[0]
	return prediction
    



In [59]:
#Get exchange rates
#ActualData = pd.read_csv('./s&p_2015-2019_daily.csv',index_col='Date')['Close']
ActualData=Data['AAPL']
ActualData=pd.DataFrame(ActualData)
ActualData.columns=['Close']
ActualData.tail()

,Close
Date,
2016-01-04,105.35
2016-01-05,102.71
2016-01-06,100.70
2016-01-07,96.45
2016-01-08,96.96


In [62]:
#Get exchange rates
#ActualData = pd.read_csv('./s&p_2015-2019_daily.csv',index_col='Date')['Close']
ActualData=Data['AAPL']
#Size of exchange rates
NumberOfElements = len(ActualData)

#Use 70% of data as training, rest 30% to Test model
TrainingSize = int(NumberOfElements * 0.7)
TrainingData = ActualData[0:TrainingSize]
TestData = ActualData[TrainingSize:NumberOfElements]

#new arrays to store actual and predictions
Actual = [x for x in TrainingData]
Predictions = list()


#in a for loop, predict values using ARIMA model
for timepoint in range(len(TestData)):
	ActualValue =  TestData[timepoint]
	#forcast value
	Prediction = StartARIMAForecasting(Actual, 3,1,1)    
	#print('Actual=%f, Predicted=%f' % (ActualValue, Prediction))
	#add it in the list
	Predictions.append(Prediction)
	Actual.append(ActualValue)

In [123]:
def MNum_Arima(logo):
    ActualData=Data[logo]
    NumberOfElements = len(ActualData)
    TrainingSize = int(NumberOfElements * 0.7)
    TrainingData = ActualData[0:TrainingSize]
    TestData = ActualData[TrainingSize:NumberOfElements]
    
    #new arrays to store actual and predictions
    Actual = [x for x in TrainingData]
    Predictions = list()
    
    
    #in a for loop, predict values using ARIMA model
    for timepoint in range(len(TestData)):
        ActualValue =  TestData[timepoint]
        #forcast value
        Prediction = StartARIMAForecasting(Actual, 3,1,1)    
        #print('Actual=%f, Predicted=%f' % (ActualValue, Prediction))
        #add it in the list
        Predictions.append(Prediction)
        Actual.append(ActualValue)
        
    return [TestData,Predictions]
        
    
    

In [178]:
#Prédiction est une liste d'array à 1 element)

def conversion(Predictions):
    L=[]
    Error=
    for k in range(len(Predictions)):
        L.append(Predictions[k][0])
    return L

In [257]:
df_ARIMA_pred2=pd.DataFrame()

error=[]
for l in logo:
    try:
        result=MNum_Arima(l)
        df_ARIMA_pred2[l+'_Pred']=conversion(result[1])
    except:
        error.append(l)
   

Problème quand ARIMA ne converge pas (SVD did not converge) le programme tourne longtemps avant de s'en rendre compte)

In [278]:
def remove_error(L,L_error):
    if len(error)==0:
        return L
    else:
        for l in L_error:
            L.remove(l)
    return L
    

In [284]:
logo_se=logo.copy()
remove_error(logo_se,error) ;#à faire tourner que une fois

In [294]:
df_ARIMA_pred=df_ARIMA_pred2
df_ARIMA_pred.tail(3).reset_index(drop=True)

,AAPL_Pred,ABT_Pred,ACN_Pred,ATVI_Pred,ADBE_Pred,GOOGL_Pred,GOOG_Pred,AMZN_Pred,AAL_Pred,BLK_Pred,CBS_Pred,MSFT_Pred,FB_Pred
0,102.849469,43.104821,103.049687,37.424541,92.701094,764.233552,745.321301,640.930369,40.372553,334.254492,46.143625,55.152921,103.474411
1,100.998130,42.808513,102.576268,37.114829,91.367640,761.609154,746.007753,640.485129,41.343079,329.934949,46.057104,54.058193,103.575202
2,96.705969,41.577668,99.740070,36.416567,89.429808,741.206175,726.555262,608.499052,40.291241,315.656298,45.291148,52.084540,97.544629


In [293]:
Data3.reset_index()

,Date,AAPL,ABT,ACN,ATVI,ADBE,AMD,GOOGL,GOOG,AMZN,AAL,BLK,CBS,MSFT,FB
0,2016-01-06,100.70,42.56,102.16,36.79,91.02,2.505,759.33,743.62,632.65,41.23,330.16,46.13,54.05,102.97
1,2016-01-07,96.45,41.54,99.16,36.27,89.11,2.275,741.00,726.39,607.94,40.45,315.73,45.34,52.17,97.92
2,2016-01-08,96.96,40.67,98.20,35.71,87.85,2.140,730.91,714.47,607.05,40.37,307.78,46.46,52.33,97.33


In [298]:
df_ARIMA_all=pd.DataFrame()
Data3=Data.tail(3).reset_index()
Arima3=df_ARIMA_pred3=df_ARIMA_pred.tail(3).reset_index(drop=True)

df_ARIMA_all['Date']=Data3['Date']

for l in logo_se:
    df_ARIMA_all[l]=Data3[l]
    df_ARIMA_all[l+'_ARIMA']=Arima3[l+'_Pred']
    
df_ARIMA_all=df_ARIMA_all.set_index('Date')


In [299]:
df_ARIMA_all

,AAPL,AAPL_ARIMA,ABT,ABT_ARIMA,ACN,ACN_ARIMA,ATVI,ATVI_ARIMA,ADBE,ADBE_ARIMA,...,AAL,AAL_ARIMA,BLK,BLK_ARIMA,CBS,CBS_ARIMA,MSFT,MSFT_ARIMA,FB,FB_ARIMA
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,100.70,102.849469,42.56,43.104821,102.16,103.049687,36.79,37.424541,91.02,92.701094,...,41.23,40.372553,330.16,334.254492,46.13,46.143625,54.05,55.152921,102.97,103.474411
2016-01-07,96.45,100.998130,41.54,42.808513,99.16,102.576268,36.27,37.114829,89.11,91.367640,...,40.45,41.343079,315.73,329.934949,45.34,46.057104,52.17,54.058193,97.92,103.575202
2016-01-08,96.96,96.705969,40.67,41.577668,98.20,99.740070,35.71,36.416567,87.85,89.429808,...,40.37,40.291241,307.78,315.656298,46.46,45.291148,52.33,52.084540,97.33,97.544629


In [300]:
df_ARIMA_all.to_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\ARIMA_All_and_Pred.csv")
